## Read generic/ALL-SKY SFR catalogues

In [1]:
import numpy as np
from astropy.table import Table, join, vstack, hstack, Column, MaskedColumn, unique
import warnings
import os, glob, getpass, sys
from astropy.utils.exceptions import AstropyWarning
from astropy import units as u
from astropy.coordinates import SkyCoord  # High-level coordinates

user = getpass.getuser()
sys.path.append('/Users/' + user + '/Dropbox/my_python_packages')

In [2]:
# Read Porras+2003 SFR source cat & add coordinates ========
read_cat = True
if read_cat:
    warnings.filterwarnings('ignore', category=AstropyWarning, append=True)
    cat    = Table.read('2003_porras.vot')
    cat.convert_bytestring_to_unicode()
    coords = [SkyCoord((cat['RAJ2000'][i] +  cat['DEJ2000'][i]), unit = (u.hourangle, u.degree)) for i in range(len(cat))]

    cat['RAJ2000_d'] = [inp.ra.deg  for inp in coords]
    cat['DEJ2000_d'] = [inp.dec.deg for inp in coords]
    for inp in ['RAJ2000_d', 'DEJ2000_d']:
        cat[inp].unit   = u.degree
        cat[inp].format = '8.5f'

cat.remove_columns(['GNum', 'SNum', 'l_Nst', 'Nst', 'e_Nst', 'r_RAh', 'EDist', 'r_Dist', 'r_Nst'])
cat.group_by('RAJ2000_d')

Name,RAJ2000,DEJ2000,Dist,RAJ2000_d,DEJ2000_d
,"""h:m:s""","""d:m:s""",pc,deg,deg
str27,str8,str9,int16,float64,float64
NGC 7822,00 03 59,+68 34 41,850,0.99583,68.57806
Mac H12,00 07 02,+65 38 39,850,1.75833,65.64417
VX Cas,00 31 30,+61 58 51,760,7.87500,61.98083
RNO 1B,00 36 47,+63 28 59,850,9.19583,63.48306
MBM 12,02 56 03,+20 06 01,275,44.01250,20.10028
AFGL 490,03 27 39,+58 46 58,900,51.91250,58.78278
NGC 1333,03 28 54,+31 19 19,350,52.22500,31.32194
IC 348,03 44 30,+32 17 00,320,56.12500,32.28333


In [3]:
# Compute distance between cluster centers
# This might be useful for future studies
cat['nearest_dist']        = 0.0 * len(cat)
cat['nearest_dist'].unit   = u.degree
cat['nearest_dist'].format = '5.2f'
cat['nearest_name']        = cat['Name']


for i in range(len(cat)):
    center_0     = SkyCoord(cat['RAJ2000_d'][i]  * u.degree, cat['DEJ2000_d'][i] * u.degree, frame = 'icrs')
    decs         = iter(cat['DEJ2000_d'])
    name         = iter(cat['Name'])
    seps, names  = [], []

    for ra in cat['RAJ2000_d']:
        coords_0 = SkyCoord(ra  * u.degree, next(decs) * u.degree, frame = 'icrs')
        seps.append(coords_0.separation(center_0).degree)
        names.append(next(name))

    tab = Table([seps, names], names = ['nearest_dist', 'nearest_name'])
    tab = tab.group_by('nearest_dist')

    cat['nearest_dist'][i] = tab['nearest_dist'][1]
    cat['nearest_name'][i] = tab['nearest_name'][1]

In [4]:
# Read Ribas catalogue =====================================
cat_2 = Table.read('2015_Ribas.csv', format = 'ascii.csv')
cat_2 = cat_2.group_by('RA')
regs  = [reg for reg in set(cat_2['Region'])]

# Compute mean coordinates for each SFR ====================
print(f'{"Region":<7s}{"Mean RA":>15s}{"Mean DEC":>10s}')
print('-'*(7+15+10))
for reg in regs:
    ra_vals  = cat_2['RA'][cat_2['Region'] == reg]
    dec_vals = cat_2['Dec'][cat_2['Region'] == reg] 
    print(f'{reg:<7s}{np.mean(ra_vals):>15.2f}{np.mean(dec_vals):>10.2f}')
      

Region         Mean RA  Mean DEC
--------------------------------
LOri             83.84      9.89
BPI             206.09    -29.58
CAR             132.79    -61.84
CrA             284.16    -36.68
TWA             175.30    -35.91
ChaI            167.13    -77.01
SOri             84.71     -2.58
NGC1333          52.28     31.34
25Ori            81.41      1.65
Serpens         277.38      0.75
ARG             134.34    -54.10
USco            241.86    -22.06
Taurus           67.73     25.80
COL              76.36    -31.30
THA              99.48    -53.38
OCT             160.03    -57.53
ABD             154.86    -25.56
Oph             246.78    -24.42
ChaII           196.09    -77.38
Lupus           238.43    -38.76
ECH             169.59    -76.43
IC348            56.13     32.14


In [5]:
#Print OPH members =========================================
cat_2[cat_2['Region'] == 'Oph']

Name,RA,Dec,Region,IRAC1,IRAC2,IRAC3,IRAC4,MIPS1,WISE1,WISE2,WISE3,WISE4
str29,float64,float64,str7,float64,float64,float64,float64,float64,float64,float64,float64,float64
Erickson2011_1,246.163116455,-24.1193199158,Oph,0.0175,0.011,0.00776,0.00494,--,0.01836,0.01048,0.004562,--
Erickson2011_2,246.165405273,-24.5858879089,Oph,0.0343,0.0258,0.0178,0.0107,--,0.03466,0.02452,0.01028,0.02493
Erickson2011_3,246.168838501,-24.71941185,Oph,0.0405,0.0259,0.0192,0.0117,--,0.04367,0.0257,0.004857,0.01335
Erickson2011_4,246.169433594,-24.3612785339,Oph,0.0407,0.0256,0.0188,0.0119,--,0.0444,0.02533,0.01472,0.02838
Erickson2011_5,246.171020508,-24.2969017029,Oph,0.11,0.0654,0.0525,0.0321,0.00282,0.1167,0.06692,0.0407,0.0666
Erickson2011_6,246.182922363,-24.7984466553,Oph,0.0569,0.0365,0.027,0.0162,0.00138,0.064,0.03568,0.006014,--
Erickson2011_7,246.186904907,-25.0056858063,Oph,--,--,--,--,--,0.004801,0.003296,--,--
Erickson2011_8,246.195007324,-24.3725166321,Oph,0.248,0.159,0.125,0.0744,0.00678,0.2901,0.1567,0.04492,0.07343
Erickson2011_9,246.195251465,-24.3672657013,Oph,0.174,0.106,0.0825,0.0515,0.00389,0.2003,0.1083,0.04262,0.08022


In [6]:
# Derive Galactic Coordinates; useful for ADQL query the Gaia Synthetic sample
# 
# Ophiuchus center in Galactic Coordinates
c_icrs = SkyCoord(ra=246*u.degree, dec=-24*u.degree, frame='icrs')
print(c_icrs.galactic)

# Gaia Background center =================
c_icrs = SkyCoord(ra=260*u.degree, dec=-5*u.degree, frame='icrs')
print(c_icrs.galactic)

<SkyCoord (Galactic): (l, b) in deg
    (352.99918914, 17.59237962)>
<SkyCoord (Galactic): (l, b) in deg
    (17.46325958, 17.73882391)>
